# Plagiarism Detection using Transformers based models   


In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
# Step 1: Install Kaggle API
!pip install kaggle

# Step 2: Upload the kaggle.json file
#from google.colab import files
#files.upload()

# Step 3: Move the kaggle.json file to the correct location
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Step 4: Download the output files from the Kaggle notebook
!kaggle kernels output danielwolffram/cord-19-create-dataframe -p /content

# Step 5: Verify the downloaded files
import os

# List the files in the destination directory
for file_name in os.listdir('/content'):
    print(file_name)

# Step 6: Load the correct CSV file into a Pandas DataFrame
import pandas as pd

# Load the correct CSV file (cord19_df.csv in this case)
df = pd.read_csv('/content/cord19_df.csv')
print(df.head())


Output file downloaded to /content/cord19_df.csv
Kernel log downloaded to /content/cord-19-create-dataframe.log 
.config
cord-19-create-dataframe.log
cord19_df.csv
kaggle.json
sample_data
                                   paper_id  \
0  3cdc48bb9e40afd30a59463b7872761a726998c8   
1  d99acb4e99be7852aa61a688c9fbd38d44b5a252   
2  748d4c57fe1acc8d9d97cf574f7dea5296f9386c   
3  b891efc6e1419713b05ff7d89b26d260478c28df   
4  353852971069ad5794445e5c1ab6077ce23da75d   

                                           body_text  \
0  NDV (Roakin strain) was obtained from Dr. D. J...   
1  Live attenuated viruses have been developed an...   
2  Ebola virus (EBOV) and other members of the fa...   
3  To the Editor:\nChina has the world's second l...   
4  Coronavirus disease 2019 (COVID-19) has spread...   

                                             methods  \
0  NDV (Roakin strain) was obtained from Dr. D. J...   
1  RSV A2 strain was obtained from ATCC (Manassas...   
2  U2OS human osteosarco

<ipython-input-4-cd6cf3edf0d1>:27: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/cord19_df.csv')


In [ ]:
source_data = pd.read_csv("cord19_df.csv", low_memory=False)

In [ ]:
def preprocess_data(data_path, sample_size):

  # Read the data from specific path
  data = pd.read_csv(data_path, low_memory=False)

  # Drop articles without Abstract
  data = data.dropna(subset = ['abstract']).reset_index(drop = True)

  # Get "sample_size" random articles
  data = data.sample(sample_size)[['abstract', 'paper_id']]

  return data


data_path = "cord19_df.csv"
source_data = preprocess_data(data_path, 100)

In [ ]:
source_data.sample(5)

,abstract,paper_id
35821,An outbreak of canine infectious respiratory d...,372f3a4aac120be6b70f0b3d51481ab2679a6b92
12708,Summary This study proposes an optimization mo...,15c19f46707c28aed4ae8bdea50b835555a5e9e0
36042,The COVID-19 pandemic is challenging modern ra...,dd8474ea0eca19eef31da0b805dbc17048d3fc11
24715,"Porcine respiratory coronavirus (PRCV), a spik...",a90439df5b69098701baa518f0a82a9e40c9bf57
22091,Abstract Emerging and infectious diseases have...,acc9960f29bf77b9c7d05dbcb78fd22df7017cbb


## Load BERT Model

In [ ]:
!pip -q install transformers
import torch
from keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer,  AutoModelForSequenceClassification

# Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
model_path = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(model_path,
                                          do_lower_case=True)

model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                          output_attentions=False,
                                                          output_hidden_states=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
import torch
from tensorflow.keras.preprocessing.sequence import pad_sequences

def create_vector_from_text(tokenizer, model, text, MAX_LEN=510):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    input_ids = tokenizer.encode(
        text,
        add_special_tokens=True,
        max_length=MAX_LEN,
        truncation=True
    )

    results = pad_sequences([input_ids], maxlen=MAX_LEN, dtype="long",
                            truncating="post", padding="post")

    # Remove the outer list.
    input_ids = results[0]

    # Create attention masks
    attention_mask = [int(i > 0) for i in input_ids]

    # Convert to tensors and move them to the appropriate device.
    input_ids = torch.tensor(input_ids).to(device)
    attention_mask = torch.tensor(attention_mask).to(device)

    # Add an extra dimension for the "batch" (even though there is only one input in this batch).
    input_ids = input_ids.unsqueeze(0)
    attention_mask = attention_mask.unsqueeze(0)

    # Move the model to the appropriate device.
    model.to(device)

    # Put the model in "evaluation" mode, meaning feed-forward operation.
    model.eval()

    # Run the text through the model and collect the hidden states.
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True  # Ensure return as dictionary
        )
        # Extract hidden states from the output dictionary.
        # Adjust this based on your model's output structure.
        encoded_layers = outputs.hidden_states

    layer_i = -1  # The last BERT layer
    batch_i = 0  # Only one input in the batch
    token_i = 0  # The first token, corresponding to [CLS]

    # Extract the embedding.
    vector = encoded_layers[layer_i][batch_i][token_i]

    # Move to the CPU and convert to numpy ndarray.
    vector = vector.cpu().numpy()

    return vector


# Create vector Database

In [ ]:
from tqdm import tqdm
import numpy as np

def create_vector_index(data):

    # The list of all the vectors
    vectors = []

    # Get overall text data
    source_data = data.abstract.values

    # Loop over all the comment and get the embeddings
    for text in tqdm(source_data):

        # Get the embedding
        vector = create_vector_from_text(tokenizer, model, text)

        #add it to the list
        vectors.append(vector)

    data["vectors"] = vectors
    data["vectors"] = data["vectors"].apply(lambda emb: np.array(emb))
    data["vectors"] = data["vectors"].apply(lambda emb: emb.reshape(1, -1))

    return data

vector_index = create_vector_index(source_data)

100%|██████████| 100/100 [00:07<00:00, 13.41it/s]


In [ ]:
vector_index.sample(5)

,abstract,paper_id,vectors
8915,Abstract Objectives To describe the pharmacy a...,9e6ffd14818d14859915ed0f26c5d8bfb93af158,"[[-0.7158729, -0.23511255, 0.06787831, -0.3705..."
20837,Abstract The carbohydrate contents of coronavi...,eac150173aec2d126e80d933621ebcb4f67f1968,"[[-0.8081973, -0.08894916, -0.50034773, -0.729..."
13372,Viruses with RNA genomes dominate the eukaryot...,4be1c96386ee9f1750f185e5f790425215cc0075,"[[-0.63432664, -0.47420996, -0.26204073, -0.95..."
4132,Abstract Introduction : The main target of COV...,3fd7868a52df3ae502c64a8318f45eb38a483913,"[[-0.7795183, -0.29794586, -0.18738492, -0.336..."
7541,Abstract Emerging and well-known viral disease...,6e49c916b5a03127c6af714fc505fa6fce312665,"[[-0.6461855, -0.56027, -0.55448836, -0.523595..."


# Language detector and translation

In [ ]:
!pip -q install sentencepiece
from transformers import MarianMTModel, MarianTokenizer

**Note**:
Make sure to restart your kernel after running the previous command to be able to use sentencepiece module.

In [ ]:
"""
Candidate Languages
de: German
fr: French      el: Greek
ja: Japan       ru: Russian
"""
language_list = ['de', 'fr', 'el', 'ja', 'ru']

In [ ]:
# Install the library
!pip -q install langdetect
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
def translate_text(text, text_lang, target_lang='en'):

  # Get the name of the model
  model_name = f"Helsinki-NLP/opus-mt-{text_lang}-{target_lang}"

  # Get the tokenizer
  tokenizer = MarianTokenizer.from_pretrained(model_name)

  # Instantiate the model
  model = MarianMTModel.from_pretrained(model_name)

  # Translation of the text
  formated_text = ">>{}<< {}".format(text_lang, text)

  translation = model.generate(**tokenizer([formated_text], return_tensors="pt", padding=True))

  translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translation][0]

  return translated_text

In [ ]:
def process_document(text):
    """
    Create a vector for given text and adjust it for cosine similarity search
    """
    text_vect = create_vector_from_text(tokenizer, model, text)
    text_vect = np.array(text_vect)
    text_vect = text_vect.reshape(1, -1)

    return text_vect


def is_plagiarism(similarity_score, plagiarism_threshold):

  is_plagiarism = False

  if(similarity_score >= plagiarism_threshold):
    is_plagiarism = True

  return is_plagiarism


def check_incoming_document(incoming_document):

  text_lang = detect(incoming_document)
  language_list = ['de', 'fr', 'el', 'ja', 'ru']

  final_result = ""

  if(text_lang == 'en'):
    final_result = incoming_document

  elif(text_lang not in language_list):
    final_result = None

  else:
    # Translate in English
    final_result = translate_text(incoming_document, text_lang)

  return final_result


def run_plagiarism_analysis(query_text, data, plagiarism_threshold=0.8):

    top_N=3

    # Check the language of the query/incoming text and translate if required.
    document_translation = check_incoming_document(query_text)

    if(document_translation is None):
      print("Only the following languages are supported: English, French, Russian, German, Greek and Japanese")
      exit(-1)

    else:
      # Preprocess the document to get the required vector for similarity analysis
      query_vect = process_document(document_translation)

      # Run similarity Search
      data["similarity"] = data["vectors"].apply(lambda x: cosine_similarity(query_vect, x))
      data["similarity"] = data["similarity"].apply(lambda x: x[0][0])

      similar_articles = data.sort_values(by='similarity', ascending=False)[0:top_N+1]
      formated_result = similar_articles[["abstract", "paper_id", "similarity"]].reset_index(drop = True)

      similarity_score = formated_result.iloc[0]["similarity"]
      most_similar_article = formated_result.iloc[0]["abstract"]
      is_plagiarism_bool = is_plagiarism(similarity_score, plagiarism_threshold)

      plagiarism_decision = {'similarity_score': round(similarity_score,4),
                             'is_plagiarism': is_plagiarism_bool,
                             'most_similar_article': most_similar_article,
                             'article_submitted': query_text
                            }

      return plagiarism_decision

In [ ]:
# Select an existing article from the database
new_incoming_text = source_data.iloc[0]['abstract']

# Run the plagiarism detection
analysis_result = run_plagiarism_analysis(new_incoming_text, vector_index, plagiarism_threshold=0.8)

In [ ]:
analysis_result

{'similarity_score': 1.0,
 'is_plagiarism': True,
 'most_similar_article': 'Abstract Glycyrrhizin is the main active compound of Glycyrrhiza glabra root extracts; according to recent studies, glycyrrhizin and its aglycon, glycyrrhetic acid, have interesting therapeutic properties. A new capillary electrophoretic method has been developed for the separation and quantification of glycyrrhizin, β-glycyrrhetic acid and its isomer α-glycyrrhetic acid. Separation of the analytes was achieved in less than 3min on a fused silica capillary, by injecting the samples at the short end of the capillary (effective length: 8.5cm). The background electrolyte was composed of pH 10.0 carbonate buffer, methanol and ethylene glycol (80/10/10) and contained 0.4% β-cyclodextrin; indomethacin was used as the internal standard. Diode array detection was used, with quantitative assays carried out at 254nm. Linearity was found over the 5–200 and 2.5–100μgmL−1 concentration ranges for glycyrrhizin and glycyrrhet

In [ ]:
french_article_to_check = """
Les Réseaux d’Innovation et de Transfert Agricole (RITA) ont été créés en 2011 pour mieux connecter la recherche et le développement agricole, intra et inter-DOM, avec un objectif d’accompagnement de la diversification des productions locales. Le CGAAER a été chargé d'analyser ce dispositif et de proposer des pistes d'action pour améliorer la chaine Recherche – Formation – Innovation – Développement – Transfert dans les outre-mer dans un contexte d'agriculture durable, au profit de l'accroissement de l'autonomie alimentaire.
"""

In [ ]:
analysis_result = run_plagiarism_analysis(french_article_to_check, vector_index, plagiarism_threshold=0.8)
analysis_result

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

{'similarity_score': 0.7736,
 'is_plagiarism': False,
 'most_similar_article': 'Antimicrobial resistance (AMR) is a global public health threat that warrants urgent attention. However, the multifaceted nature of AMR often complicates the development and implementation of comprehensive policies. In this study, we describe the policy context and explore experts’ perspectives on the challenges, facilitators, and strategies for combating AMR in Singapore. We conducted semi-structured interviews with 21 participants. Interviews were transcribed verbatim and were analyzed thematically, adopting an interpretative approach. Participants reported that the Ministry of Health (MOH) has effectively funded AMR control programs and research in all public hospitals. In addition, a preexisting One Health platform, among MOH, Agri-Food & Veterinary Authority (restructured to form the Singapore Food Agency and the Animal & Veterinary Service under NParks in April 2019), National Environment Agency, and 

In [ ]:
german_article_to_check = """Derzeit ist eine Reihe strukturell und funktionell unterschiedlicher temperaturempfindlicher Elemente wie RNA-Thermometer bekannt, die eine Vielzahl biologischer Prozesse in Bakterien, einschließlich der Virulenz, steuern. Auf der Grundlage einer Computer- und thermodynamischen Analyse der vollständig sequenzierten Genome von 25 Salmonella enterica-Isolaten wurden ein Algorithmus und Kriterien für die Suche nach potenziellen RNA-Thermometern entwickelt. Er wird es ermöglichen, die Suche nach potentiellen Riboschaltern im Genom anderer gesellschaftlich wichtiger Krankheitserreger durchzuführen. Für S. enterica wurden neben dem bekannten 4U-RNA-Thermometer vier Hairpin-Loop-Strukturen identifiziert, die wahrscheinlich als weitere RNA-Thermometer fungieren. Sie erfüllen die notwendigen und hinreichenden Bedingungen für die Bildung von RNA-Thermometern und sind hochkonservative nichtkanonische Strukturen, da diese hochkonservativen Strukturen im Genom aller 25 Isolate von S. enterica gefunden wurden. Die Hairpins, die eine kreuzförmige Struktur in der supergewickelten pUC8-DNA bilden, wurden mit Hilfe der Rasterkraftmikroskopie sichtbar gemacht."""




analysis_result = run_plagiarism_analysis(german_article_to_check, vector_index, plagiarism_threshold=0.8)
analysis_result

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

{'similarity_score': 0.8813,
 'is_plagiarism': True,
 'most_similar_article': 'Abstract The carbohydrate contents of coronavirus glycoproteins E1 and E2 have been analyzed. E2 has complex and mannose-rich-type oligosaccharide side-chains, which are attached by N-glycosidic linkages to the polypeptide. Glycosylation of E2 is initiated at the co-translational level, and it is inhibited by tunicamycin, 2-deoxy-glucose, and 2-deoxy-2-fluoro-glucose. Thus, E2 belongs to a glycoprotein type found in many other enveloped viruses. E1, in contrast, represents a different class of glycoprotein. The following observations indicate that its carbohydrate side-chains have 0-glycosidic linkage. (1) The constituent sugars of E1 are N-acetylglucosamine, N-acetylgalactosamine, galactose, and neuraminic acid; mannose and fucose are absent. (2) The side-chains can be removed by β-elimination. (3) Glycosylation of E1 is not sensitive to the compounds interfering with N-glycosylation. E1 is the first viral 